In [23]:
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
import matplotlib.pyplot as plt

In [4]:
# Here we create our simple neural network. For more details here we are subclassing and
# inheriting from nn.Module, this is the most general way to create your networks and
# allows for more flexibility. I encourage you to also check out nn.Sequential which
# would be easier to use in this scenario but I wanted to show you something that

In [181]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        """
        Here we define the layers of the network. We create two fully connected layers

        Parameters:
            input_size: the size of the input, in this case 784 (28x28)
            num_classes: the number of classes we want to predict, in this case 10 (0-9)

        """
        super().__init__()
        # Our first linear layer take input_size, in this case 784 nodes to 50
        # and our second linear layer takes 50 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        """
        x here is the mnist images and we run it through fc1, fc2 that we created above.
        we also add a ReLU activation function in between and for that (since it has no parameters)
        I recommend using nn.functional (F)

        Parameters:
            x: mnist images

        Returns:
            out: the output of the network
        """

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [183]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

In [185]:
# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [187]:
# Initialize network
model = NN(input_size=input_size, num_classes=num_classes)

In [189]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [191]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [01:59<00:00,  7.85it/s]


In [55]:
def check_accuracy(loader, model):
    """
    Check accuracy of our trained model given a loader and a model

    Parameters:
        loader: torch.utils.data.DataLoader
            A loader for the dataset you want to check accuracy on
        model: nn.Module
            The model you want to check accuracy on

    Returns:
        acc: float
            The accuracy of the model on the dataset given by the loader
    """

    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:
            # Get to correct shape
            x = x.reshape(x.shape[0], -1)

            # Forward pass 
            scores = model(x) #64 x 10 matrix 64 is the batch size, 10 is num_classes
            _, predictions = scores.max(1) #give us the maximum value of each row of the matrix and its indice, we can use scores.argmax(1) instead

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


In [57]:
# Check accuracy on training & test to see how good our model
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 96.06
Accuracy on test set: 95.44


In [105]:
for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    if batch_idx == 10:
        break
    print(type(data))
    print(data.reshape(data.shape[0], -1).shape)
    data = nn.Flatten(data)
    print(type(data))

  1%|▊                                                                               | 10/938 [00:00<00:05, 180.18it/s]

<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>
<class 'torch.Tensor'>
torch.Size([64, 784])
<class 'torch.nn.modules.flatten.Flatten'>


In [161]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        if batch_idx == 1:
            break
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # Forward
        scores = model(data)
        print(scores.shape)
        print(scores.max(1))
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()

  0%|                                                                                  | 1/938 [00:00<00:32, 29.01it/s]

torch.Size([64, 10])
torch.return_types.max(
values=tensor([ 5.8206,  6.4088,  8.2700,  6.5946,  6.9546,  8.1498,  7.8014,  8.7487,
         7.9128,  5.7195, 10.3948,  7.0461,  3.7557,  4.2113,  7.7321, 11.6473,
         6.1670,  8.8498,  9.7461,  7.6122,  7.4517,  9.4291,  6.4006,  1.9015,
         9.1702,  4.1103,  8.9394,  8.5605, 13.7045,  6.7924,  7.1509,  7.4686,
        10.7217, 10.2292,  9.9222,  5.1790, 13.0021,  7.0120,  5.8735,  7.2510,
         9.8781,  7.9612, 13.5689,  3.5592,  3.8612,  5.9975,  3.7144,  4.2430,
         7.7463,  8.8735,  5.6881,  4.2716,  9.7905,  7.2907, 10.8334,  7.5945,
         7.3691,  6.4914,  9.5999, 10.3801,  5.9047,  5.2315,  6.7468,  3.1682],
       grad_fn=<MaxBackward0>),
indices=tensor([8, 8, 9, 6, 4, 6, 3, 0, 8, 3, 2, 3, 4, 3, 4, 6, 4, 8, 0, 0, 9, 4, 6, 9,
        5, 8, 4, 0, 0, 1, 1, 4, 2, 2, 3, 0, 7, 8, 1, 1, 3, 5, 7, 1, 8, 1, 8, 4,
        8, 6, 1, 7, 0, 5, 6, 5, 3, 8, 9, 2, 0, 8, 6, 2]))


In [155]:
temp = torch.tensor([[1, 2, 3], [3, 4, 5]])

In [177]:
_, pred = temp.max(1)
pred

tensor([2, 2])

In [179]:
temp.argmax(1)

tensor([2, 2])